In [1]:
# Cell 1: imports & basic config

import json
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Any
from dotenv import load_dotenv
import os

import numpy as np

# If you're using the OpenAI client:
from openai import OpenAI

# load_dotenv()  # looks for .env in current dir
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # or use env var

# Embedding model name (adjust to what you're actually using)
EMBEDDING_MODEL = "text-embedding-3-small"

# Chat model for intent parsing
INTENT_MODEL = "gpt-4.1-mini"  # or whatever you're on


In [2]:
# Cell 2: load YANG catalog

CATALOG_PATH = "sensor_catalog.jsonl"  # or .json

all_rows = []

with open(CATALOG_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        all_rows.append(json.loads(line))

len(all_rows)


54063

In [3]:
# Cell 3: define intent schema

@dataclass
class TelemetryIntent:
    # High-level task
    task: str                      # e.g. "generate_telemetry_config"

    # Protocol / domain
    protocol_tag: Optional[str]    # e.g. "bgp", "ospf", "qos"
    vendor: Optional[str]          # e.g. "cisco"
    os: Optional[str]              # e.g. "ios xr"

    # Transport / export details
    transport: Optional[str]       # e.g. "grpc"
    tls: Optional[bool]            # True / False / None
    destination_address: Optional[str]
    destination_port: Optional[int]

    # Telemetry sampling / style
    sample_interval_ms: Optional[int]  # if user mentions "every 30 seconds"

    # What kind of data they care about
    metric_focus: List[str]       # e.g. ["neighbors", "stats", "status"]
    extra_requirements: str       # free-text notes (e.g. "no tls", "self-describing-gpb")

    # Raw original query (so you can always fall back)
    original_query: str

    # Helper: construct from dict safely
    @staticmethod
    def from_dict(d: Dict[str, Any]) -> "TelemetryIntent":
        return TelemetryIntent(
            task=d.get("task", "generate_telemetry_config"),
            protocol_tag=d.get("protocol_tag"),
            vendor=d.get("vendor"),
            os=d.get("os"),
            transport=d.get("transport"),
            tls=d.get("tls"),
            destination_address=d.get("destination_address"),
            destination_port=d.get("destination_port"),
            sample_interval_ms=d.get("sample_interval_ms"),
            metric_focus=d.get("metric_focus") or [],
            extra_requirements=d.get("extra_requirements", ""),
            original_query=d.get("original_query", ""),
        )


In [4]:
# Cell 4: build prompt for the intent LLM

INTENT_SYSTEM_PROMPT = """
You are a network telemetry assistant. Your job is to convert a natural language request
into a structured JSON object describing what telemetry configuration is needed.

Follow these rules:
- Only output valid JSON, no comments, no explanations.
- Use the following keys exactly:

{
  "task": string,                     // e.g. "generate_telemetry_config"
  "protocol_tag": string or null,     // one of: bgp, ospf, isis, mpls, ldp, multicast, routing, interfaces, qos, acl, platform, tunnel, or null
  "vendor": string or null,           // e.g. "cisco"
  "os": string or null,               // e.g. "ios xr"
  "transport": string or null,        // e.g. "grpc"
  "tls": boolean or null,             // true, false, or null if not specified
  "destination_address": string or null,
  "destination_port": integer or null,
  "sample_interval_ms": integer or null,   // telemetry sampling interval in ms if mentioned
  "metric_focus": array of strings,   // short tags like "neighbors", "routes", "stats", "summary", "status"
  "extra_requirements": string,       // any additional constraints (formats, comments, etc.)
  "original_query": string            // echo the original user query
}

If the user doesn't specify something, set it to null (or [] for arrays) instead of guessing too much.
When inferring protocol_tag, map phrases like "border gateway protocol" to "bgp".
""".strip()


In [5]:
# Cell 5: LLM call -> JSON -> TelemetryIntent

def parse_intent(query: str) -> TelemetryIntent:
    response = client.chat.completions.create(
        model=INTENT_MODEL,
        messages=[
            {"role": "system", "content": INTENT_SYSTEM_PROMPT},
            {"role": "user", "content": query},
        ],
        temperature=0.0,
    )

    content = response.choices[0].message.content
    # content should be pure JSON per instructions
    try:    # Ensure original_query is filled

        data = json.loads(content)
    except json.JSONDecodeError as e:
        raise ValueError(f"Model did not return valid JSON: {e}\n\n{content}")

    # Ensure original_query is filled
    if not data.get("original_query"):
        data["original_query"] = query

    return TelemetryIntent.from_dict(data)
    # return data


In [6]:
# Cell 6: test with your sample query

query = (
    "generate telemetry configuration for cisco ios xr about bgp protocol ? "
    "Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. "
    "Choose relevant sensor-paths."
)

intent = parse_intent(query)
intent


TelemetryIntent(task='generate_telemetry_config', protocol_tag='bgp', vendor='cisco', os='ios xr', transport='grpc', tls=False, destination_address='192.0.2.0', destination_port=57500, sample_interval_ms=None, metric_focus=['neighbors', 'routes', 'status'], extra_requirements='Use relevant BGP sensor-paths for telemetry in Cisco IOS XR.', original_query='generate telemetry configuration for cisco ios xr about bgp protocol ? Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. Choose relevant sensor-paths.')

In [7]:
# Canonical: TelemetryIntent -> text for embeddings

def intent_to_search_text(intent: TelemetryIntent) -> str:
    """
    Convert a TelemetryIntent into a rich, structured text representation
    suitable for semantic embedding and vector search over YANG sensor paths.
    """

    parts: list[str] = []

    # High-level summary first (good for embeddings)
    summary_bits = []
    if intent.protocol_tag:
        summary_bits.append(f"{intent.protocol_tag.upper()}")
    if intent.vendor:
        summary_bits.append(intent.vendor)
    if intent.os:
        summary_bits.append(intent.os)
    proto_stack = " ".join(summary_bits) if summary_bits else "unspecified protocol"

    parts.append(f"Telemetry intent: generate telemetry configuration for {proto_stack}")

    # Core fields
    parts.append(f"Task: {intent.task}")

    if intent.protocol_tag:
        parts.append(f"Protocol: {intent.protocol_tag}")

    if intent.vendor:
        parts.append(f"Vendor: {intent.vendor}")

    if intent.os:
        parts.append(f"OS: {intent.os}")

    # Transport + security
    if intent.transport or intent.tls is not None:
        if intent.tls is False:
            security_str = "no-tls"
        elif intent.tls is True:
            security_str = "tls"
        else:
            security_str = "unspecified"
        if intent.transport:
            parts.append(f"Transport: {intent.transport} ({security_str})")
        else:
            parts.append(f"Security: {security_str}")

    # Destination (collector) info
    if intent.destination_address or intent.destination_port:
        addr = intent.destination_address or "unspecified-address"
        port = intent.destination_port if intent.destination_port is not None else "unspecified-port"
        parts.append(f"Destination: {addr}:{port}")

    # Sampling interval
    if intent.sample_interval_ms:
        parts.append(f"Sample interval (ms): {intent.sample_interval_ms}")

    # What metrics we care about (matches your YANG categories nicely)
    if intent.metric_focus:
        parts.append("Metric focus: " + ", ".join(intent.metric_focus))

    # Extra requirements from the planner / user
    if intent.extra_requirements:
        parts.append("Extra requirements: " + intent.extra_requirements)

    # Always include the original query at the end; it's pure semantic gold
    if intent.original_query:
        parts.append("Original query: " + intent.original_query)

    return "\n".join(parts)


In [8]:
intent_text = intent_to_search_text(intent)
intent_text

'Telemetry intent: generate telemetry configuration for BGP cisco ios xr\nTask: generate_telemetry_config\nProtocol: bgp\nVendor: cisco\nOS: ios xr\nTransport: grpc (no-tls)\nDestination: 192.0.2.0:57500\nMetric focus: neighbors, routes, status\nExtra requirements: Use relevant BGP sensor-paths for telemetry in Cisco IOS XR.\nOriginal query: generate telemetry configuration for cisco ios xr about bgp protocol ? Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. Choose relevant sensor-paths.'

In [9]:
from openai import OpenAI
from qdrant_client.models import PointStruct

client_oa = OpenAI()  # uses OPENAI_API_KEY from env

EMBEDDING_MODEL = "text-embedding-3-small"

def get_embedding(text: str) -> list[float]:
    resp = client_oa.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text,
    )
    return resp.data[0].embedding

query_vector = get_embedding(intent_text)


In [10]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# Local Qdrant example – adjust for Cloud (host, api_key, etc.)
client = QdrantClient(host="localhost", port=6333)


if not client.collection_exists("yang_sensors"):
    client.create_collection(
        collection_name="yang_sensors",
        vectors_config={
            "yang_vector": VectorParams(size=1536, distance="Cosine")
        },
    )


In [11]:
# Cell 2: load YANG catalog
import json

CATALOG_PATH = "sensor_catalog.jsonl"  # or .json

all_rows = []

with open(CATALOG_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        all_rows.append(json.loads(line))

len(all_rows)


54063

In [12]:
from qdrant_client.models import PointStruct

points = []

for idx, row in enumerate(all_rows[:10]):
    vector = get_embedding(row["search_text"])

    point = PointStruct(
        id=idx,  # ✅ valid: unsigned integer
        vector=vector,
        payload={
            "yang_id": row["id"],          # "yang-0"
            "module": row["module"],
            "path": row["path"],
            "protocol_tag": row["protocol_tag"],
            "category": row["category"],
            "kind": row["kind"],
            "leaf_count": row["leaf_count"],
            "description": row["description"],
            "leaf_names": row["leaf_names"],
        },
    )
    points.append(point)

client.upsert(
    collection_name="yang_sensors",
    points=points,
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

query_filter = Filter(
    must=[
        FieldCondition(
            key="protocol_tag",
            match=MatchValue(value=intent.protocol_tag)
        )
    ]
)


In [14]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

points, next_page = client.scroll(
    collection_name="yang_sensors",
    with_payload=True,
    with_vectors=False,
    limit=5,
)

for p in points:
    print("id:", p.id)
    print("payload:", p.payload)
    print("protocol_tag in payload:", p.payload.get("protocol_tag"))
    print("---")


id: 0
payload: {'yang_id': 0, 'module': 'Cisco-IOS-XR-tunnel-ip-ma-oper', 'path': 'Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma', 'protocol_tag': 'tunnel', 'category': ['state', 'stats', 'tunnels'], 'kind': 'container', 'leaf_count': 92, 'description': 'Tunnel Ip ma parameters', 'leaf_names': ['address-family', 'adjacency', 'afi', 'bandwidth', 'base-caps-state', 'bfd-session-state', 'cap-ipv4-transport-supported', 'cap-ipv6-transport-supported', 'check-point-id', 'convergence-state', 'destination-address-length', 'destination-prefix-list', 'endpt-count', 'endpt-prod', 'eod-recvd', 'ep-app-id', 'flag-bits', 'flags', 'flags-other', 'gre-cap-checksum-supported', 'gre-cap-ipv4-transport-supported', 'gre-cap-ipv6-transport-supported', 'gre-cap-key-supported', 'gre-cap-max-mtu-supported', 'gre-cap-max-tunnels-supported', 'gre-cap-mgre-ipv4-transport-supported', 'gre-cap-mgre-ipv6-transport-supported', 'gre-cap-multi-encap-supported', 'gre-cap-platform-supported', 'gre-cap-seq-num-supported', 

In [18]:
hits = client.search(
    collection_name="yang_sensors",
    query_vector=query_vector,
    query_filter=query_filter,
    limit=10,
)

# No matches with protocol filter? Retry without it.
if not hits and query_filter is not None:
    hits = client.search(
        collection_name="yang_sensors",
        query_vector=query_vector,
        limit=10,
        with_payload=True,
    )


for h in hits:
    print(h.id, h.payload["yang_id"], h.payload["path"])


5 5 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data
4 4 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas
0 0 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma
3 3 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database
1 1 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/gsp-node-db-summary
2 2 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/gsp-node-db-summary/gspdb-array
6 6 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data/idb-array
7 7 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data/idb-array/source-address
8 8 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data/idb-array/destination-address
9 9 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/tunnel-ids


/tmp/ipykernel_32338/968722398.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
/tmp/ipykernel_32338/968722398.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
